Based on the code shown in https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_sift.html

In [1]:
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.feature import SIFT, match_descriptors, plot_matched_features
from skimage import transform
from PIL import Image
import numpy as np
import os
import cv2
from skimage.measure import ransac
from skimage.transform import ProjectiveTransform
from pathlib import Path
import pandas as pd

In [2]:
# Load the datafiles and make them grayscale
def load_image(path):
    return rgb2gray(np.array(Image.open(path)))

In [3]:
# Used to acces the name of the folder we are working on : not necessary
print("Répertoire courant :", os.getcwd())


Répertoire courant : c:\Users\emmar\OneDrive\Polytech\I5\Semestre 9 - Oslo\Data Mining\Exercices\Day 5\Augmented_reality_system\src


In [4]:
# Extract SIFT keypoints and descriptors
def extract_sift_features(image):
    sift = SIFT()
    sift.detect_and_extract(image)
    return sift.keypoints, sift.descriptors

Problem of dimensions for the first two pictures.

In [5]:
def resize_to_match(img1, img2):
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    return cv2.resize(img2, (w1, h1)) if (h1 != h2 or w1 != w2) else img2

In [6]:
# Visualize the matches
def match_and_plot(img1, img2, keypoints1, descriptors1, keypoints2, descriptors2, title):
    matches = match_descriptors(descriptors1, descriptors2, max_ratio=0.6, cross_check=True)

    #img2 = resize_to_match(img1, img2)
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    plt.gray()
    plot_matched_features(image0=img1, image1=img2, 
                          keypoints0=keypoints1, keypoints1=keypoints2,
                          matches=matches, ax=ax, 
                          keypoints_color='cyan',
                          only_matches=True)
    ax.axis('off')
    ax.set_title(title)
    plt.show()
    return matches

In [17]:
# Compute matching accuracy using optional homography
def compute_matching_accuracy(keypoints1, keypoints2, matches, homography=None, threshold=5):
    """
    Compute the matching accuracy based on a homography.
    If no homography is provided, only the number of matches is printed.
    """
    if homography is None:
        print(f"No homographt matrix to work with")
        return None

    if matches is None or len(matches) == 0:
        print("No matches to evaluate.")
        return 0

    correct = 0
    for i, j in matches:
        pt1 = np.array([*keypoints1[i], 1.0])  # homogeneous coordinates
        projected = homography @ pt1
        projected /= projected[2]  # normalize

        pt2 = keypoints2[j]
        error = np.linalg.norm(projected[:2] - pt2)

        if error < threshold:
            correct += 1

    accuracy = correct / len(matches) if matches.size > 0 else 0
    print(f"Matching Accuracy: {accuracy:.2%} ({correct}/{len(matches)} correct matches)")
    return accuracy

In [15]:
def create_homography(keypoints1, keypoints2, descriptors1, descriptors2):
    """
    Create a homography matrix from source points to destination points.
    """
    matches = match_descriptors(descriptors1, descriptors2, max_ratio=0.6, cross_check=True)
    if matches.shape[0] < 4:
        print(f"Not enough matches to estimate homography: {matches.shape[0]} found")
        return None, None
    
    if matches is None:
        print("No matches found.")
        return None, None
    
    src = keypoints1[matches[:, 0]]  # points from image 1
    dst = keypoints2[matches[:, 1]]  # corresponding points in image 2

    model_robust, inliers = ransac(
        (src, dst),
        ProjectiveTransform,
        min_samples=4,
        residual_threshold=2,
        max_trials=1000
    )
    H = model_robust.params 
    return H, matches


In [9]:
def load_base(dataset_dir = '../data/images'):
    dataset_paths = sorted(Path(dataset_dir).rglob("*"))
    records = []

    for ref_path in dataset_paths:
        print(f"Currently working on {ref_path}")
        
        if ref_path.suffix.lower() not in [".jpg", ".jpeg", ".png", ".avif"]:
            continue
            
        img_ref = load_image(str(ref_path))
        keypoints_ref, descriptors_ref = extract_sift_features(img_ref)

        if len(keypoints_ref) == 0:
            print(f" Skipped {ref_path.name} due to missing keypoints")
            continue

        record = {
            "filename" : ref_path.name,
            "path": str(ref_path),
            "num_keypoints": len(keypoints_ref),
            "keypoints": keypoints_ref,
            "descriptors": descriptors_ref,
            "image_shape": img_ref.shape,
            "image": img_ref,
        }
        records.append(record)

    df = pd.DataFrame(records)
    return df

In [10]:
data = load_base()

Currently working on ..\data\images\city_hall
Currently working on ..\data\images\city_hall\city_hall1.jpg
Currently working on ..\data\images\city_hall\city_hall2.jpg
Currently working on ..\data\images\city_hall\city_hall3.jpeg
Currently working on ..\data\images\city_hall\city_hall4.jpg
Currently working on ..\data\images\fram
Currently working on ..\data\images\fram\fram1.png
Currently working on ..\data\images\fram\fram2.jpeg
Currently working on ..\data\images\fram\fram3.jpeg
Currently working on ..\data\images\fram\fram4.jpg
Currently working on ..\data\images\fram\fram5.avif
Currently working on ..\data\images\opera
Currently working on ..\data\images\opera\opera1.jpg
Currently working on ..\data\images\opera\opera2.jpg
Currently working on ..\data\images\opera\opera3.jpeg
Currently working on ..\data\images\opera\opera4.jpg
Currently working on ..\data\images\opera\opera5.jpg
Currently working on ..\data\images\palace
Currently working on ..\data\images\palace\palace1.jpg
Curr

In [ ]:
dataset_paths = sorted(Path('../data/test').rglob("*"))


for ref_path in dataset_paths:
    if ref_path.suffix.lower() not in [".jpg", ".jpeg", ".png", ".avif"]:
        continue
    
    print(f"Working on {ref_path}")

    img_ref = load_image(str(ref_path))
    keypoints_ref, descriptors_ref = extract_sift_features(img_ref)

    for _, files in data.iterrows():
        print(f"File from the dataset : {files["path"]}")
        H1, matches = create_homography(files["keypoints"], keypoints_ref, files["descriptors"], descriptors_ref)
        print(f"Accuracy of Image vs Image test : {compute_matching_accuracy(files['keypoints'], keypoints_ref, matches, H1)}")

    

Working on ..\data\test\city_hall_test.jpg
File from the dataset : ..\data\images\city_hall\city_hall1.jpg
Matching Accuracy: 18.18% (4/22 correct matches)
Accuracy of Image vs Image test : 0.18181818181818182
File from the dataset : ..\data\images\city_hall\city_hall2.jpg
Not enough matches to estimate homography: 2 found
No homographt matrix to work with
Accuracy of Image vs Image test : None
File from the dataset : ..\data\images\city_hall\city_hall3.jpeg
Matching Accuracy: 100.00% (5/5 correct matches)
Accuracy of Image vs Image test : 1.0
File from the dataset : ..\data\images\city_hall\city_hall4.jpg
Matching Accuracy: 56.25% (9/16 correct matches)
Accuracy of Image vs Image test : 0.5625
File from the dataset : ..\data\images\fram\fram1.png
Not enough matches to estimate homography: 1 found
No homographt matrix to work with
Accuracy of Image vs Image test : None
File from the dataset : ..\data\images\fram\fram2.jpeg
Not enough matches to estimate homography: 0 found
No homograph

In [12]:
#matches = match_and_plot(img1, im_test, keypoints1, descriptors1, keypoints_test, descriptors_test, "Image 1 vs Image test")
#H1, matches = create_homography(keypoints1, keypoints_test, descriptors1, descriptors_test)
#print(f"Accurcay of Image 1 vs Image test : {compute_matching_accuracy(keypoints1, keypoints_test, matches, H1)}")